**Author:** Prof. AJ Smit

Department of Biodiversity and Conservation Biology

University of the Western Cape

# Topic 14: Cluster analysis

In this example we will apply two types of cluster analysis. **K-means clustering** and **hierarchical clustering**. Whereas ordination attempts to display the presence and influence of gradients, clustering tries to place our samples into a certain number of discrete units or clusters.

## Set-up the analysis environment

In [5]:
library(tidyverse) 
library(GGally)
library(cluster)
library(purrr)
library(dendextend)
library(ggcorrplot)
library(DataExplorer)
library(factoextra)
library(gridExtra)
library(vegan)

Loading required package: permute


Attaching package: ‘permute’


The following object is masked from ‘package:dendextend’:

    shuffle


Loading required package: lattice

Registered S3 method overwritten by 'vegan':
  method     from      
  rev.hclust dendextend

This is vegan 2.5-7



## Country data

Data taken from a [Kaggle challenge](https://www.kaggle.com/rohan0301/unsupervised-learning-on-country-data). Information about the data and the analysis is repeated here almost verbatim, but I have included various edits and modifications to the analysis.

### Clustering the Countries by using Unsupervised Learning for HELP International
#### Objective
To categorise the countries using socio-economic and health factors that determine the overall development of the country.

#### About organization
HELP International is an international humanitarian NGO that is committed to fighting poverty and providing the people of backward countries with basic amenities and relief during the time of disasters and natural calamities.

#### Problem Statement
HELP International have been able to raise around $ 10 million. Now the CEO of the NGO needs to decide how to use this money strategically and effectively. So, CEO has to make decision to choose the countries that are in the direst need of aid. Hence, your Job as a Data scientist is to categorise the countries using some socio-economic and health factors that determine the overall development of the country. Then you need to suggest the countries which the CEO needs to focus on the most.

## Load the data and examine it

In [3]:
data <- read_csv("/Users/ajsmit/Dropbox/R/workshops/Quantitative_Ecology/_GitBook/data/Country-data.csv")
head(data)
summary(data)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  country = col_character(),
  child_mort = col_double(),
  exports = col_double(),
  health = col_double(),
  imports = col_double(),
  income = col_double(),
  inflation = col_double(),
  life_expec = col_double(),
  total_fer = col_double(),
  gdpp = col_double()
)




country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200
Argentina,14.5,18.9,8.10,16.0,18700,20.90,75.8,2.37,10300


   country            child_mort        exports            health      
 Length:167         Min.   :  2.60   Min.   :  0.109   Min.   : 1.810  
 Class :character   1st Qu.:  8.25   1st Qu.: 23.800   1st Qu.: 4.920  
 Mode  :character   Median : 19.30   Median : 35.000   Median : 6.320  
                    Mean   : 38.27   Mean   : 41.109   Mean   : 6.816  
                    3rd Qu.: 62.10   3rd Qu.: 51.350   3rd Qu.: 8.600  
                    Max.   :208.00   Max.   :200.000   Max.   :17.900  
    imports             income         inflation         life_expec   
 Min.   :  0.0659   Min.   :   609   Min.   : -4.210   Min.   :32.10  
 1st Qu.: 30.2000   1st Qu.:  3355   1st Qu.:  1.810   1st Qu.:65.30  
 Median : 43.3000   Median :  9960   Median :  5.390   Median :73.10  
 Mean   : 46.8902   Mean   : 17145   Mean   :  7.782   Mean   :70.56  
 3rd Qu.: 58.7500   3rd Qu.: 22800   3rd Qu.: 10.750   3rd Qu.:76.80  
 Max.   :174.0000   Max.   :125000   Max.   :104.000   Max.   :82.80  

See the data descriptors:

In [2]:
read_csv("/Users/ajsmit/Dropbox/R/workshops/Quantitative_Ecology/_GitBook/data/data-dictionary.csv")


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  `Column Name` = col_character(),
  Description = col_character()
)




Column Name,Description
<chr>,<chr>
country,Name of the country
child_mort,Death of children under 5 years of age per 1000 live births
exports,Exports of goods and services per capita. Given as %age of the GDP per capita
health,Total health spending per capita. Given as %age of GDP per capita
imports,Imports of goods and services per capita. Given as %age of the GDP per capita
Income,Net income per person
Inflation,The measurement of the annual growth rate of the Total GDP
life_expec,The average number of years a new born child would live if the current mortality patterns are to remain the same
total_fer,The number of children that would be born to each woman if the current age-fertility rates remain the same.


The data are in different measurement scales, so standardise it:

In [6]:
data_std <- decostand(data[, 2:10], method = "standardize")

## Do the clustering